## 03 Decisioning

Copper Day Trader - Daily Directional Trading



### Imports

In [11]:
import numpy as np
import pandas as pd
import ibis
import matplotlib.pyplot as plt
import methods.prep as prep
import methods.vis as vis
import methods.fc as fc
import methods.sim as sim
import importlib

### Load Data

In [93]:
con = ibis.connect("duckdb://")
data = ibis.read_csv('data_forecasting/data.csv')

Dev/Test Split

In [94]:
# Split the data into model development (training + validation) (2007-01-01 to 2019-12-31) 
dev_data = data.filter(data.DATE.year() >= 2007).filter(data.DATE.year() <= 2019)

# and holdout test set (2020-01-01 to the end of the dataset 
# in late 2024, plus 2019 padding to be dropped later)
test_data = data.filter(data.DATE.year() >= 2019)

### Run Forecasting & Enrichment - Development

In [95]:
dev_df = dev_data.to_pandas()
dev_df['DATE'] = pd.to_datetime(dev_df['DATE'])
dev_df = dev_df.sort_values('DATE')

In [50]:
test_df = test_data.to_pandas()
test_df['DATE'] = pd.to_datetime(test_df['DATE'])
test_df = test_df.sort_values('DATE')

In [98]:
# Forecast
importlib.reload(fc)
dev_df = fc.sliding_window_arima_predictions(
    df = dev_df,
    target_name= 'COPPER_OPEN_NOMINAL',
    pdq = (1,2,1),
    window_size=12)

C:\Users\haits\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\haits\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\haits\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:

In [99]:
# Enrichment
# Add several columns based on the prediction target
# to aid in evaluation and decisioning
importlib.reload(fc)
dev_df = fc.add_fc_eval_columns(
    df = dev_df,
    pred_feature = 'COPPER_OPEN_NOMINAL')

In [100]:
columns = [
    'DATE',
    'COPPER_OPEN_NOMINAL',
    'COPPER_OPEN_NOMINAL_PRED',
    'COPPER_OPEN_NOMINAL_DELTA',
    'COPPER_OPEN_NOMINAL_DELTA_PRED',
    'COPPER_OPEN_NOMINAL_PROPDELTA',
    'COPPER_OPEN_NOMINAL_PROPDELTA_PRED']

dev_df[columns]

,DATE,COPPER_OPEN_NOMINAL,COPPER_OPEN_NOMINAL_PRED,COPPER_OPEN_NOMINAL_DELTA,COPPER_OPEN_NOMINAL_DELTA_PRED,COPPER_OPEN_NOMINAL_PROPDELTA,COPPER_OPEN_NOMINAL_PROPDELTA_PRED
0,2007-01-02,2.8710,NaN,NaN,NaN,NaN,NaN
1,2007-01-03,2.7980,NaN,-0.0730,NaN,-0.025427,NaN
2,2007-01-04,2.6600,NaN,-0.1380,NaN,-0.049321,NaN
3,2007-01-05,2.6110,NaN,-0.0490,NaN,-0.018421,NaN
4,2007-01-08,2.5350,NaN,-0.0760,NaN,-0.029108,NaN
...,...,...,...,...,...,...,...
3292,2019-12-25,2.8250,2.805038,0.0235,0.003538,0.008388,0.001263
3293,2019-12-26,2.8370,2.828721,0.0120,0.003721,0.004248,0.001317
3294,2019-12-27,2.8465,2.843904,0.0095,0.006904,0.003349,0.002433
3295,2019-12-30,2.8245,2.850128,-0.0220,0.003628,-0.007729,0.001274


In [101]:
#Limit to 2008-2019 due to NaNs; drop remaining NaNs if any
dev_df = dev_df[dev_df['DATE'].dt.year >= 2008].dropna()

Creat buckets for predicted price deltas for the decisioning model to act on

In [106]:
def value_to_bin(x : float,
             breakpoints : list)->int:

    bin = 0
    for breakpoint in breakpoints:
        if x < breakpoint:
            return bin
        bin = bin + 1
    return bin

def values_to_bins(
        df : pd.DataFrame,
        breakpoints : list,
        target_col : str)->pd.DataFrame:
    
    df[target_col + '_BIN'] = df[target_col].apply(lambda x : value_to_bin(x, breakpoints))
    return df

In [104]:
# Calculate breakpoints to divide predicted proportional deltas into buckets according to quantiles
breakpoints = list(dev_df['COPPER_OPEN_NOMINAL_PROPDELTA_PRED'].quantile([x/11.0 for x in range(1,11)]))
breakpoints

[-0.010425720164294371,
 -0.005947391526572783,
 -0.0037235149764711013,
 -0.0021274785085398868,
 -0.0007188654360384516,
 0.0005797337313629594,
 0.0020048888592230775,
 0.003829380022559951,
 0.0061611199439798375,
 0.010490247373252676]

In [107]:
# Add a column with the new bins
dev_df = values_to_bins(dev_df,
    breakpoints,
    'COPPER_OPEN_NOMINAL_PROPDELTA_PRED',)

columns = [
    'DATE',
    'COPPER_OPEN_NOMINAL_PROPDELTA_PRED',
    'COPPER_OPEN_NOMINAL_PROPDELTA_PRED_BIN']

dev_df[columns]

,DATE,COPPER_OPEN_NOMINAL_PROPDELTA_PRED,COPPER_OPEN_NOMINAL_PROPDELTA_PRED_BIN
252,2008-01-02,-0.023190,0
253,2008-01-03,-0.013525,0
254,2008-01-04,-0.000047,5
255,2008-01-07,0.030516,10
256,2008-01-08,0.014530,10
...,...,...,...
3292,2019-12-25,0.001263,6
3293,2019-12-26,0.001317,6
3294,2019-12-27,0.002433,7
3295,2019-12-30,0.001274,6


In [431]:
importlib.reload(sim)

agent = sim.PortfolioAgent(
    data = dev_df[columns],
    date_col= 'DATE',
    price_delta_pred_bins_col = 'COPPER_OPEN_NOMINAL_PROPDELTA_PRED_BIN',
    price_delta_col = 'COPPER_OPEN_NOMINAL_PROPDELTA_PRED',
    learning_rate = 1, 
    explore_chance = 0.3, #Chance to take a random (legal) action
    rebalance_limit_steps = 2,  # Determine how far asset balances can be changed with each action
    asset_balance_steps = [x/10.0 for x in range(11)],  # Possible asset balances; 0 is all cash, 1 is all copper futures
)

In [1067]:
agent.step(
    exploring=True,
    learning=True
)

print(str(agent.current_step))
print(str(agent.asset_balance_at_open_ind[agent.current_step]))
print(str(agent.portfolio_value[agent.current_step]))

636
2
919865.1006313763
